In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.spatial import distance_matrix

In [ ]:
img_name = 'monza' # Input image name
im_path = img_name + '.png' # make have to change im type

In [ ]:
def get_grayscale(im_path): 
    image = cv2.imread(im_path) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

# plt.imshow(gray, cmap = 'gray')
# plt.show()

In [ ]:
# Credit: https://pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/
def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

In [ ]:
def sort_by_nearest_distance_fast(points):
    """Sort points by nearest distance using SciPy's distance_matrix."""
    if not points:
        return []
    
    # Convert to NumPy array for efficient computation
    points = np.array(points)
    sorted_points = [points[0]]  # Start with the first point
    points = np.delete(points, 0, axis=0)  # Remove the first point from the list

    while len(points) > 0:
        # Compute distances from the last sorted point to all remaining points
        last_point = sorted_points[-1]
        distances = np.linalg.norm(points - last_point, axis=1)
        
        # Find the index of the nearest point
        nearest_index = np.argmin(distances)
        sorted_points.append(points[nearest_index])
        
        # Remove the nearest point from the remaining points
        points = np.delete(points, nearest_index, axis=0)
    
    return np.array(sorted_points).tolist()

In [ ]:
def get_edges_list(edges):
    # Inner and outer edges list
    out_edges = []
    in_edges = []
    
    # Find contours, hierarchy
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    # Iterate through contours
    outer_edge_indx = -1 # 'First Parent'
    max_hier = np.max(hierarchy[:, :, -1]) # 'Youngest Child'
    
    # Find the maximum value from the final index
    for i, h in enumerate(hierarchy[0]):
        if h[3] == -1:  # Outer edge (no parent)
            if outer_edge_indx == -1:
                outer_edge_indx = i
                out_edges.append(contours[i].reshape(-1, 2).tolist())
    for i, h in enumerate(hierarchy[0]):
        if h[3] == max_hier:  # Inner edge (has a parent)
            in_edges.append(contours[i].reshape(-1, 2).tolist())

    in_edges = sort_by_nearest_distance_fast(in_edges)
    in_edges[0].append(in_edges[0][0])

    out_edges = sort_by_nearest_distance_fast(out_edges)
    out_edges[0].append(out_edges[0][0])

    return out_edges, in_edges

In [ ]:
def show_edges(out_edges, in_edges):
    for i, point_list in enumerate(in_edges):
        in_x = [point[0] for point in point_list]  # Extract x-coordinates
        in_y = [-point[1] for point in point_list]  # Extract y-coordinates
    for i, point_list in enumerate(out_edges):
        out_x = [point[0] for point in point_list]  # Extract x-coordinates
        out_y = [-point[1] for point in point_list]  # Extract y-coordinates
    
    plt.plot(in_x, in_y, color='red', label = 'Inner Edge')
    plt.plot(out_x, out_y, color='blue', label = 'Outer Edge')
    plt.legend()

    plt.show()

In [ ]:
def create_dictionary(out_edges, in_edges):
    # Data to be written
    name = "tfrecord-" + img_name + ".json"
    roads_dictionary = []

    # Base template for road edges
    road_edge_dict = {
             "geometry":[],
             "type": "road_edge"
             }
    
    # Base template for road line
    road_line_dict = {
             "geometry":[],
             "type": "road_line"
             }
    
    # Base Template for Object JSON
    object_dict = {
        "position":[],
        "width": None,
        "length": None,
        "heading":[],
        "velocity":[],
        "valid":[],
        "goalPosition": {
            "x": None,
            "y": None,
        },
        "type": "vehicle",
    }
    
    # ------------- ROADS ----------------------
    
    # Copies for inner and outer edges
    in_edge_dict = {
             "geometry":[],
             "type": "road_edge"
             }
    out_edge_dict = {
             "geometry":[],
             "type": "road_edge"
             }
    
    # Add outer edge points
    for out_indx in out_edges[0]:
        out_edge_dict["geometry"].append({"x": out_indx[0], "y": out_indx[1]})
    
    # Add inner edge points
    for in_indx in in_edges[0]:
        in_edge_dict["geometry"].append({"x": in_indx[0], "y": in_indx[1]})
    
    roads_dictionary.append(out_edge_dict)
    roads_dictionary.append(in_edge_dict)
    
    # ------------- POSITION ----------------------
    nearest_coords = []
    widths = []
    out_np = np.array(out_edges[0])
    in_np = np.array(in_edges[0])
    
    # Compute distances of given point on outer edge to all points on inner edge
    for coord in out_np:
        distances = np.linalg.norm(in_np - coord, axis=1)  
        nearest_index = np.argmin(distances)  # Find the index of the nearest coordinate
        nearest_coords.append(in_np[nearest_index].tolist())  # Append the nearest coordinate
        widths.append(distances[nearest_index]) # Append the nearest width
    
    min_indx = 91 # Hard-coded frame rate equivalent to Waymo Open Dataset
    
    # Position the vehicle at midpoint of track
    x_list = []
    y_list = []
    for i in range(min_indx):
        x = (out_edges[0][i][0] + nearest_coords[i][0]) / 2
        y = (out_edges[0][i][1] + nearest_coords[i][1]) / 2
        x_list.append(x)
        y_list.append(y)
        object_dict["position"].append({"x": x, "y": y})
        object_dict["valid"].append(True)
    
    # Heading
    
    # Velocity
    rate = 1
    for i in range(min_indx):
        if i != (min_indx - 1):
            vel_x = (x_list[i + 1] - x_list[i]) * rate
            vel_y = (y_list[i + 1] - y_list[i]) * rate
        else: # Ending velocity is (0, 0)
            vel_x = 0
            vel_y = 0
        object_dict["velocity"].append({"x": vel_x, "y": vel_y})
        
    # Goal position
    object_dict["goalPosition"]["x"] = x_list[min_indx - 1]
    object_dict["goalPosition"]["y"] = y_list[min_indx - 1]
    
    # Vehicle Dimensions
    track_width = np.min(widths)
    object_dict["width"] = track_width * 0.3
    object_dict["length"] = track_width * 0.6

    #------------- ROAD LINE -------------------
    # Add "optimal line" as road line, originating with midpoint of width
    opt_line = road_line_dict = {
             "geometry":[],
             "type": "road_line"
             }
    for i in range(len(out_edges[0])):
        x = (out_edges[0][i][0] + nearest_coords[i][0]) / 2
        y = (out_edges[0][i][1] + nearest_coords[i][1]) / 2
        opt_line["geometry"].append({"x": x, "y": y})
        
    roads_dictionary.append(opt_line)
    
    
    # ------------ CREATE FULL DICTIONARY ------------
    full_dictionary = {
        "name": name,
        "objects": [],
        "roads": roads_dictionary,
        "tl_states": {}
    }
    full_dictionary["objects"].append(object_dict)
    return full_dictionary

In [ ]:
# Calculating an optimal line using gradient descent


In [ ]:
def serialize(full_dictionary, name):
    # Serializing json
    json_object = json.dumps(full_dictionary, indent=4)
    
    # Specify the folder path and file name
    folder_path = "/home/ar3015/IW/gpudrive/data/processed/f1/"
    file_path = os.path.join(folder_path, "tfrecord-" + name + ".json")
    
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)
    # Writing to sample.json
    with open(file_path, "w") as outfile:
        outfile.write(json_object)

In [ ]:
# Put all functions together
def create_json(im_path, name):
    print("Serialized ", im_path)
    gray = get_grayscale(im_path)
    edges = auto_canny(gray)
    out_edges, in_edges = get_edges_list(edges)
    show_edges(out_edges, in_edges)
    full_dictionary = create_dictionary(out_edges, in_edges)
    serialize(full_dictionary, name)

In [ ]:
directory = "/home/ar3015/IW/gpudrive/F1_23_tracks/"
for filename in os.listdir(directory):
    name = filename.split(".")[0]
    path = os.path.join(directory, filename)
    if os.path.isfile(path):
        create_json(path, name)